In [1]:
import pandas as pd
import re
import openpyxl
import streamlit as st
from io import BytesIO
from datetime import datetime
import plotly.express as px
import matplotlib.pyplot as plt
import requests

In [2]:
url = "https://webservice-d.jfce.jus.br/sarh_new/json/buscarMapaCorregedoria"
try:
    resposta = requests.get(url, timeout=20)
    resposta.raise_for_status()
    dados_json = resposta.json()
    # Acessa a lista de funcionários dentro do dicionário
    funcionarios = dados_json["mapaCorregedoria"]["funcionarios"]
    # Converte para DataFrame
    dados = pd.DataFrame(funcionarios)
except requests.exceptions.RequestException as e:
    st.write(f"Erro ao acessar o webservice: {e}. Comunique a equipe responsável.")

In [5]:
dados.head()

,matricula,nome,codigoLotacao,descricaoLotacao,codigoLotacaoPai,descricaoLotacaoPai,codigoCargo,descricaoCargo,provimento,divisao,situacao,tipoPerfil,classe,nivel,padrao,vinculo
0,CE1769,FLAVIA RABELLO DE MOURA,10124,SECAO DE APOIO ADMINISTRATIVO E JUDICIARIO,5801,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,341.0,TÉCNICO JUDICIÁRIO/ ADMINISTRATIVA (AGENTE DE ...,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,B,NI,9,EFETIVO
1,CE1744,GARCIAS ALENCAR FILHO,10126,SETOR TECNICO DE APOIO JUDICIARIO,5801,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,295.0,ANALISTA JUDICIÁRIO/ JUDICIÁRIA,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,B,NS,9,EFETIVO
2,CE1633,JOSE WILTON EVANGELISTA FREIRES,10125,SETOR TECNICO DE APOIO ADMINISTRATIVO,5801,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,316.0,TÉCNICO JUDICIÁRIO/ ADMINISTRATIVA,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,C,NI,12,EFETIVO
3,CE1984,ANDRESSA TORRES BESSA,10134,SETOR TECNICO DE APOIO ADMINISTRATIVO,5804,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,297.0,ANALISTA JUDICIÁRIO/ ADMINISTRATIVA,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,A,NS,1,EFETIVO
4,CE1936,LIA MARA SOBRAL BRITO SILVEIRA,10135,SETOR TECNICO DE APOIO JUDICIARIO,5804,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,295.0,ANALISTA JUDICIÁRIO/ JUDICIÁRIA,REDISTRIBUICAO,SERVIDOR DO QUADRO,ATIVO,SA,C,NS,13,EFETIVO


In [4]:
def definir_vinculo(row):
    if row["divisao"] == "SERVIDOR DO QUADRO":
        return "EFETIVO"
    elif row["divisao"] == "SEM VINCULO":
        return "SEM VÍNCULO"
    elif row["divisao"] == "SERVIDOR DE OUTROS ORGAOS":
        if row["provimento"] == "AUTORIZACAO PARA EXERCICIO PROVISORIO":
            return "EXERCÍCIO PROVISÓRIO"
        elif "REMOV" in row["situacao"]:
            return "REMOVIDO"
        else:
            return "REQUISITADOS"
    else:
        return "OUTRO"
dados["vinculo"] = dados.apply(definir_vinculo, axis=1)
dados.head()

,matricula,nome,codigoLotacao,descricaoLotacao,codigoLotacaoPai,descricaoLotacaoPai,codigoCargo,descricaoCargo,provimento,divisao,situacao,tipoPerfil,classe,nivel,padrao,vinculo
0,CE1769,FLAVIA RABELLO DE MOURA,10124,SECAO DE APOIO ADMINISTRATIVO E JUDICIARIO,5801,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,341.0,TÉCNICO JUDICIÁRIO/ ADMINISTRATIVA (AGENTE DE ...,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,B,NI,9,EFETIVO
1,CE1744,GARCIAS ALENCAR FILHO,10126,SETOR TECNICO DE APOIO JUDICIARIO,5801,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,295.0,ANALISTA JUDICIÁRIO/ JUDICIÁRIA,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,B,NS,9,EFETIVO
2,CE1633,JOSE WILTON EVANGELISTA FREIRES,10125,SETOR TECNICO DE APOIO ADMINISTRATIVO,5801,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,316.0,TÉCNICO JUDICIÁRIO/ ADMINISTRATIVA,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,C,NI,12,EFETIVO
3,CE1984,ANDRESSA TORRES BESSA,10134,SETOR TECNICO DE APOIO ADMINISTRATIVO,5804,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,297.0,ANALISTA JUDICIÁRIO/ ADMINISTRATIVA,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,A,NS,1,EFETIVO
4,CE1936,LIA MARA SOBRAL BRITO SILVEIRA,10135,SETOR TECNICO DE APOIO JUDICIARIO,5804,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,295.0,ANALISTA JUDICIÁRIO/ JUDICIÁRIA,REDISTRIBUICAO,SERVIDOR DO QUADRO,ATIVO,SA,C,NS,13,EFETIVO


In [7]:
cargo_order = [
    "ANALISTA JUDICIÁRIO/ ADMINISTRATIVA",
    "ANALISTA JUDICIÁRIO/ APOIO ESPECIALIZADO (ANÁLISE DE SISTEMAS DE INFORMAÇÃO)",
    "ANALISTA JUDICIÁRIO/ APOIO ESPECIALIZADO (BIBLIOTECONOMIA)",
    "ANALISTA JUDICIÁRIO/ APOIO ESPECIALIZADO (CONTABILIDADE)",
    "ANALISTA JUDICIÁRIO/ APOIO ESPECIALIZADO (ENGENHARIA (CIVIL))",
    "ANALISTA JUDICIÁRIO/ APOIO ESPECIALIZADO (INFORMÁTICA (INFRAESTRUTURA))",
    "ANALISTA JUDICIÁRIO/ APOIO ESPECIALIZADO (MEDICINA (CLÍNICA GERAL))",
    "ANALISTA JUDICIÁRIO/ APOIO ESPECIALIZADO (PSICOLOGIA)",
    "ANALISTA JUDICIÁRIO/ APOIO ESPECIALIZADO (TECNOLOGIA DA INFORMAÇÃO)",
    "ANALISTA JUDICIÁRIO/ JUDICIÁRIA",
    "ANALISTA JUDICIÁRIO/ JUDICIÁRIA (OFICIAL DE JUSTIÇA AVALIADOR FEDERAL)",
    "TÉCNICO JUDICIÁRIO/ ADMINISTRATIVA",
    "TÉCNICO JUDICIÁRIO/ ADMINISTRATIVA (AGENTE DE POLÍCIA JUDICIAL)",
    "TÉCNICO JUDICIÁRIO/ APOIO ESPECIALIZADO (CONTABILIDADE)",
    "TÉCNICO JUDICIÁRIO/ APOIO ESPECIALIZADO (TECNOLOGIA DA INFORMAÇÃO)",
    "SEM DESCRIÇÃO DE CARGO"]
dados['descricaoCargo'] = dados['descricaoCargo'].apply(lambda x: "SEM DESCRIÇÃO DE CARGO" if pd.isna(x) or x == "" else x)
dados['descricaoCargo'] = pd.Categorical(dados['descricaoCargo'], categories=cargo_order, ordered=True)
dados.head()

,matricula,nome,codigoLotacao,descricaoLotacao,codigoLotacaoPai,descricaoLotacaoPai,codigoCargo,descricaoCargo,provimento,divisao,situacao,tipoPerfil,classe,nivel,padrao,vinculo
0,CE1769,FLAVIA RABELLO DE MOURA,10124,SECAO DE APOIO ADMINISTRATIVO E JUDICIARIO,5801,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,341.0,TÉCNICO JUDICIÁRIO/ ADMINISTRATIVA (AGENTE DE ...,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,B,NI,9,EFETIVO
1,CE1744,GARCIAS ALENCAR FILHO,10126,SETOR TECNICO DE APOIO JUDICIARIO,5801,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,295.0,ANALISTA JUDICIÁRIO/ JUDICIÁRIA,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,B,NS,9,EFETIVO
2,CE1633,JOSE WILTON EVANGELISTA FREIRES,10125,SETOR TECNICO DE APOIO ADMINISTRATIVO,5801,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,316.0,TÉCNICO JUDICIÁRIO/ ADMINISTRATIVA,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,C,NI,12,EFETIVO
3,CE1984,ANDRESSA TORRES BESSA,10134,SETOR TECNICO DE APOIO ADMINISTRATIVO,5804,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,297.0,ANALISTA JUDICIÁRIO/ ADMINISTRATIVA,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,A,NS,1,EFETIVO
4,CE1936,LIA MARA SOBRAL BRITO SILVEIRA,10135,SETOR TECNICO DE APOIO JUDICIARIO,5804,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,295.0,ANALISTA JUDICIÁRIO/ JUDICIÁRIA,REDISTRIBUICAO,SERVIDOR DO QUADRO,ATIVO,SA,C,NS,13,EFETIVO


In [8]:
def substituir_gabinete(lotacao):
    if lotacao == "GABINETE DO JUIZ FEDERAL DIRETOR DO FORO":
        return "DIRETORIA DO FORO"
    elif lotacao == "GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO DE LIMOEIRO DO NORTE":
        return "SUBDIRETORIA DO FORO - LIMOEIRO DO NORTE"
    elif lotacao == "GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO DE JUAZEIRO DO NORTE":
        return "SUBDIRETORIA DO FORO - JUAZEIRO DO NORTE"
    elif lotacao == "GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO DE SOBRAL":
        return "SUBDIRETORIA DO FORO - SOBRAL"
    elif lotacao == "GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO DE CRATEUS":
        return "SUBDIRETORIA DO FORO - CRATEÚS"
    elif lotacao == "GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO DE QUIXADA":
        return "SUBDIRETORIA DO FORO - QUIXADÁ"
    elif lotacao == "GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO DE TAUA":
        return "SUBDIRETORIA DO FORO - TAUÁ"
    elif lotacao == "GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO DE IGUATU":
        return "SUBDIRETORIA DO FORO - IGUATU"
    elif lotacao == "GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO DE ITAPIPOCA":
        return "SUBDIRETORIA DO FORO - ITAPIPOCA"
    elif lotacao == "GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO-MARACANAU-CE":
        return "SUBDIRETORIA DO FORO - MARACANAÚ"
    else:
        return lotacao

dados['descricaoLotacaoPai_subdiretoria'] = dados['descricaoLotacaoPai'].apply(substituir_gabinete)
dados.head()

,matricula,nome,codigoLotacao,descricaoLotacao,codigoLotacaoPai,descricaoLotacaoPai,codigoCargo,descricaoCargo,provimento,divisao,situacao,tipoPerfil,classe,nivel,padrao,vinculo,descricaoLotacaoPai_subdiretoria
0,CE1769,FLAVIA RABELLO DE MOURA,10124,SECAO DE APOIO ADMINISTRATIVO E JUDICIARIO,5801,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,341.0,TÉCNICO JUDICIÁRIO/ ADMINISTRATIVA (AGENTE DE ...,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,B,NI,9,EFETIVO,SUBDIRETORIA DO FORO - CRATEÚS
1,CE1744,GARCIAS ALENCAR FILHO,10126,SETOR TECNICO DE APOIO JUDICIARIO,5801,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,295.0,ANALISTA JUDICIÁRIO/ JUDICIÁRIA,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,B,NS,9,EFETIVO,SUBDIRETORIA DO FORO - CRATEÚS
2,CE1633,JOSE WILTON EVANGELISTA FREIRES,10125,SETOR TECNICO DE APOIO ADMINISTRATIVO,5801,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,316.0,TÉCNICO JUDICIÁRIO/ ADMINISTRATIVA,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,C,NI,12,EFETIVO,SUBDIRETORIA DO FORO - CRATEÚS
3,CE1984,ANDRESSA TORRES BESSA,10134,SETOR TECNICO DE APOIO ADMINISTRATIVO,5804,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,297.0,ANALISTA JUDICIÁRIO/ ADMINISTRATIVA,NOMEACAO (CONCURSO PUBLICO),SERVIDOR DO QUADRO,ATIVO,SA,A,NS,1,EFETIVO,SUBDIRETORIA DO FORO - IGUATU
4,CE1936,LIA MARA SOBRAL BRITO SILVEIRA,10135,SETOR TECNICO DE APOIO JUDICIARIO,5804,GABINETE DO JUIZ FEDERAL DIRETOR DA SUBSEÇAO D...,295.0,ANALISTA JUDICIÁRIO/ JUDICIÁRIA,REDISTRIBUICAO,SERVIDOR DO QUADRO,ATIVO,SA,C,NS,13,EFETIVO,SUBDIRETORIA DO FORO - IGUATU


In [ ]:
grupos_subsecao = {
    'TOTAL FORTALEZA': [
        "1ª VARA - FORTALEZA-CE","2ª VARA - FORTALEZA-CE","3ª VARA - FORTALEZA-CE","4ª VARA - FORTALEZA-CE",
        "5ª VARA - FORTALEZA-CE","6ª VARA - FORTALEZA-CE","7ª VARA - FORTALEZA-CE","8ª VARA - FORTALEZA-CE",
        "9ª VARA - FORTALEZA-CE","10ª VARA - FORTALEZA-CE","11ª VARA - FORTALEZA-CE","12ª VARA - FORTALEZA-CE",
        "13ª VARA - JEF - FORTALEZA-CE","14ª VARA - JEF - FORTALEZA-CE","20ª VARA - FORTALEZA-CE",
        "21ª VARA - JEF - FORTALEZA-CE","26ª VARA - JEF - FORTALEZA - CE","28ª VARA - JEF - FORTALEZA-CE",
        "32ª VARA - FORTALEZA-CE","33ª VARA - FORTALEZA-CE",
        "1ª TURMA RECURSAL","2ª TURMA RECURSAL","3ª TURMA RECURSAL",
        "SECRETARIA ADMINISTRATIVA","DIRETORIA DO FOROO"],
    "TOTAL RECURSAIS": ["1ª TURMA RECURSAL","2ª TURMA RECURSAL","3ª TURMA RECURSAL"],
    'TOTAL LIMOEIRO DO NORTE': ["15ª VARA - LIMOEIRO DO NORTE-CE","29ª VARA - JEF - LIMOEIRO DO NORTE - CE",
                        "SUBDIRETORIA DO FORO - LIMOEIRO DO NORTE"],
    'TOTAL JUAZEIRO DO NORTE': ["16ª VARA - JUAZEIRO DO NORTE-CE","17ª VARA - JEF - JUAZEIRO DO NORTE-CE",
                        "30ª VARA - JEF - JUAZEIRO DO NORTE - CE",
                        "SUBDIRETORIA DO FORO - JUAZEIRO DO NORTE"],
    'TOTAL SOBRAL': ["18ª VARA - SOBRAL-CE","19ª VARA - JEF - SOBRAL-CE","31ª VARA - JEF - SOBRAL - CE",
                    "SUBDIRETORIA DO FORO - SOBRAL"],
    'TOTAL CRATEÚS': ["22ª VARA - CRATEÚS-CE","SUBDIRETORIA DO FORO - CRATEÚS"],
    'TOTAL QUIXADÁ': ["23ª VARA - QUIXADÁ-CE","SUBDIRETORIA DO FORO - QUIXADÁ"],
    'TOTAL TAUÁ': ["24ª VARA - TAUÁ-CE","SUBDIRETORIA DO FORO - TAUÁ"],
    'TOTAL IGUATU': ["25ª VARA - IGUATU-CE","SUBDIRETORIA DO FORO - IGUATU"],
    'TOTAL ITAPIPOCA': ["27ª VARA- ITAPIPOCA-CE","SUBDIRETORIA DO FORO - ITAPIPOCA"],
    'TOTAL MARACANAÚ': ["34ª VARA - MARACANAÚ-CE","35ª VARA - JEF - MARACANAÚ-CE",
                        "SUBDIRETORIA DO FORO - MARACANAÚ"],
    'TOTAL SERVIDORES COM LOTAÇÃO': ['TOTAL FORTALEZA','TOTAL LIMOEIRO DO NORTE''TOTAL JUAZEIRO DO NORTE','TOTAL SOBRAL','TOTAL CRATEÚS',
                        'TOTAL QUIXADÁ','TOTAL TAUÁ','TOTAL IGUATU','TOTAL ITAPIPOCA', 'TOTAL MARACANAÚ'],
    'TOTAL SERVIDORES SEM LOTAÇÃO': ["SERVIDORA EM LICENÇA GESTANTE EXERC. FUNÇÃO",
                    "SERVIDORES CEDIDOS/EXERCICIO PROVISÓRIO/REMOVIDO"],
    'TOTAL SERVIDORES': ['TOTAL SERVIDORES COM LOTAÇÃO','TOTAL SERVIDORES SEM LOTAÇÃO'],
    'TOTAL NÚCLEOS': ["NUCLEO DE AUDITORIA INTERNA","NUCLEO DE ESTRATEGIA, GOVERNANÇA E INTEGRIDADE",
                    "NUCLEO DE GESTAO DE PESSOAS","NUCLEO DE GESTAO ORCAMENTARIA FINANCEIRA CONTABIL E PATRIMONIAL",
                    "NUCLEO DE INFRAESTRUTURA E ADMINISTRAÇAO PREDIAL","NUCLEO DE INTELIGENCIA, SEGURANÇA E TRANSPORTE",
                    "NUCLEO DE TECNOLOGIA DA INFORMAÇAO E COMUNICAÇAO","NUCLEO JUDICIARIO"],
}

In [ ]:
ordem_colunas_mapa  = [
    "1ª VARA - FORTALEZA-CE","2ª VARA - FORTALEZA-CE","3ª VARA - FORTALEZA-CE","4ª VARA - FORTALEZA-CE",
    "5ª VARA - FORTALEZA-CE","6ª VARA - FORTALEZA-CE","7ª VARA - FORTALEZA-CE","8ª VARA - FORTALEZA-CE",
    "9ª VARA - FORTALEZA-CE","10ª VARA - FORTALEZA-CE","11ª VARA - FORTALEZA-CE","12ª VARA - FORTALEZA-CE",
    "13ª VARA - JEF - FORTALEZA-CE","14ª VARA - JEF - FORTALEZA-CE","20ª VARA - FORTALEZA-CE",
    "21ª VARA - JEF - FORTALEZA-CE","26ª VARA - JEF - FORTALEZA - CE","28ª VARA - JEF - FORTALEZA-CE",
    "32ª VARA - FORTALEZA-CE","33ª VARA - FORTALEZA-CE",
    "1ª TURMA RECURSAL","2ª TURMA RECURSAL","3ª TURMA RECURSAL","TOTAL RECURSAIS",
    "SECRETARIA ADMINISTRATIVA","DIRETORIA DO FORO",
    "TOTAL FORTALEZA",

    "15ª VARA - LIMOEIRO DO NORTE-CE","29ª VARA - JEF - LIMOEIRO DO NORTE - CE",
    "SUBDIRETORIA DO FORO - LIMOEIRO DO NORTE",'TOTAL LIMOEIRO DO NORTE',
    
    "16ª VARA - JUAZEIRO DO NORTE-CE","17ª VARA - JEF - JUAZEIRO DO NORTE-CE",
    "30ª VARA - JEF - JUAZEIRO DO NORTE - CE",
    "SUBDIRETORIA DO FORO - JUAZEIRO DO NORTE",'TOTAL JUAZEIRO DO NORTE',
    
    "18ª VARA - SOBRAL-CE","19ª VARA - JEF - SOBRAL-CE","31ª VARA - JEF - SOBRAL - CE",
    "SUBDIRETORIA DO FORO - SOBRAL",'TOTAL SOBRAL',
    
    "22ª VARA - CRATEÚS-CE","SUBDIRETORIA DO FORO - CRATEÚS",'TOTAL CRATEÚS',

    "23ª VARA - QUIXADÁ-CE","SUBDIRETORIA DO FORO - QUIXADÁ",'TOTAL QUIXADÁ',

    "24ª VARA - TAUÁ-CE","SUBDIRETORIA DO FORO - TAUÁ",'TOTAL TAUÁ',

    "25ª VARA - IGUATU-CE","SUBDIRETORIA DO FORO - IGUATU",'TOTAL IGUATU',

    "27ª VARA- ITAPIPOCA-CE","SUBDIRETORIA DO FORO - ITAPIPOCA",'TOTAL ITAPIPOCA',

    "34ª VARA - MARACANAÚ-CE","35ª VARA - JEF - MARACANAÚ-CE",
    "SUBDIRETORIA DO FORO - MARACANAÚ",'TOTAL MARACANAÚ',
    
    'TOTAL SERVIDORES COM LOTAÇÃO',

    "SERVIDORA EM LICENÇA GESTANTE EXERC. FUNÇÃO","SERVIDORES CEDIDOS/EXERCICIO PROVISÓRIO/REMOVIDO",
    'TOTAL SERVIDORES SEM LOTAÇÃO',
    'TOTAL SERVIDORES',
    
    "NUCLEO DE AUDITORIA INTERNA","NUCLEO DE ESTRATEGIA, GOVERNANÇA E INTEGRIDADE",
    "NUCLEO DE GESTAO DE PESSOAS","NUCLEO DE GESTAO ORCAMENTARIA FINANCEIRA CONTABIL E PATRIMONIAL",
    "NUCLEO DE INFRAESTRUTURA E ADMINISTRAÇAO PREDIAL","NUCLEO DE INTELIGENCIA, SEGURANÇA E TRANSPORTE",
    "NUCLEO DE TECNOLOGIA DA INFORMAÇAO E COMUNICAÇAO","NUCLEO JUDICIARIO","TOTAL NÚCLEOS"]

ordem_abas = [
    "1ª VARA - FORTALEZA-CE","2ª VARA - FORTALEZA-CE","3ª VARA - FORTALEZA-CE","4ª VARA - FORTALEZA-CE",
    "5ª VARA - FORTALEZA-CE","6ª VARA - FORTALEZA-CE","7ª VARA - FORTALEZA-CE","8ª VARA - FORTALEZA-CE",
    "9ª VARA - FORTALEZA-CE","10ª VARA - FORTALEZA-CE","11ª VARA - FORTALEZA-CE","12ª VARA - FORTALEZA-CE",
    "13ª VARA - JEF - FORTALEZA-CE","14ª VARA - JEF - FORTALEZA-CE","20ª VARA - FORTALEZA-CE",
    "21ª VARA - JEF - FORTALEZA-CE","26ª VARA - JEF - FORTALEZA - CE","28ª VARA - JEF - FORTALEZA-CE",
    "32ª VARA - FORTALEZA-CE","33ª VARA - FORTALEZA-CE",
    "1ª TURMA RECURSAL","2ª TURMA RECURSAL","3ª TURMA RECURSAL",
    "SECRETARIA ADMINISTRATIVA","DIRETORIA DO FORO",

    "15ª VARA - LIMOEIRO DO NORTE-CE","29ª VARA - JEF - LIMOEIRO DO NORTE - CE",
    "SUBDIRETORIA DO FORO - LIMOEIRO DO NORTE",
    
    "16ª VARA - JUAZEIRO DO NORTE-CE","17ª VARA - JEF - JUAZEIRO DO NORTE-CE",
    "30ª VARA - JEF - JUAZEIRO DO NORTE - CE",
    "SUBDIRETORIA DO FORO - JUAZEIRO DO NORTE",
    
    "18ª VARA - SOBRAL-CE","19ª VARA - JEF - SOBRAL-CE","31ª VARA - JEF - SOBRAL - CE",
    "SUBDIRETORIA DO FORO - SOBRAL",
    
    "22ª VARA - CRATEÚS-CE","SUBDIRETORIA DO FORO - CRATEÚS",

    "23ª VARA - QUIXADÁ-CE","SUBDIRETORIA DO FORO - QUIXADÁ",

    "24ª VARA - TAUÁ-CE","SUBDIRETORIA DO FORO - TAUÁ",'TOTAL TAUÁ',

    "25ª VARA - IGUATU-CE","SUBDIRETORIA DO FORO - IGUATU",

    "27ª VARA- ITAPIPOCA-CE","SUBDIRETORIA DO FORO - ITAPIPOCA",

    "34ª VARA - MARACANAÚ-CE","35ª VARA - JEF - MARACANAÚ-CE",
    "SUBDIRETORIA DO FORO - MARACANAÚ",

    "SERVIDORA EM LICENÇA GESTANTE EXERC. FUNÇÃO","SERVIDORES CEDIDOS/EXERCICIO PROVISÓRIO/REMOVIDO",
    
    "NUCLEO DE AUDITORIA INTERNA","NUCLEO DE ESTRATEGIA, GOVERNANÇA E INTEGRIDADE",
    "NUCLEO DE GESTAO DE PESSOAS","NUCLEO DE GESTAO ORCAMENTARIA FINANCEIRA CONTABIL E PATRIMONIAL",
    "NUCLEO DE INFRAESTRUTURA E ADMINISTRAÇAO PREDIAL","NUCLEO DE INTELIGENCIA, SEGURANÇA E TRANSPORTE",
    "NUCLEO DE TECNOLOGIA DA INFORMAÇAO E COMUNICAÇAO","NUCLEO JUDICIARIO"]
